<a href="https://colab.research.google.com/github/MarinaChau/IASD_classes/blob/master/Part_2_Single_cell_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install libraries
!pip install scprep
!pip install tensorflow==1.12

ERROR: Could not find a version that satisfies the requirement tensorflow==1.12 (from versions: 1.13.1, 1.13.2, 1.14.0, 1.15.0, 1.15.2, 1.15.3, 1.15.4, 1.15.5, 2.0.0, 2.0.1, 2.0.2, 2.0.3, 2.0.4, 2.1.0, 2.1.1, 2.1.2, 2.1.3, 2.1.4, 2.2.0rc0, 2.2.0rc1, 2.2.0rc2, 2.2.0rc3, 2.2.0rc4, 2.2.0, 2.2.1, 2.2.2, 2.2.3, 2.3.0rc0, 2.3.0rc1, 2.3.0rc2, 2.3.0, 2.3.1, 2.3.2, 2.3.3, 2.3.4, 2.4.0rc0, 2.4.0rc1, 2.4.0rc2, 2.4.0rc3, 2.4.0rc4, 2.4.0, 2.4.1, 2.4.2, 2.4.3, 2.4.4, 2.5.0rc0, 2.5.0rc1, 2.5.0rc2, 2.5.0rc3, 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.8.0rc0, 2.8.0rc1, 2.8.0)
ERROR: No matching distribution found for tensorflow==1.12


# Part 2. Analyses of datasets using unsupervised approach

This is a dataset describing development of a dentate gyrus, a part of the mouse brain. Each object is a brain cell
characterized by the expression of ~10000 genes. The cells are labeled into 24 distinct cell types.

 In this notebook, we will use different libraries to visualize our dataset.

In [ ]:
from google.colab import drive

try:
  drive._mount('/content/drive') 
except Exception:
  drive.mount('/content/drive')


Mounted at /content/drive


In [66]:
## Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import io
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
### Third-party libraries

## Exercice
* 1) Choose one of the datasets 1-3 or suggest you own
* 2) Check if you have missing values and suggest a strategy for imputing them
* 3) Check if the dataset needs to be normalized in a particular way
* 4) Give an estimate of intrinsic dimensionality of the dataset (if you want you can use skdim package
* https://github.com/j-bac/scikit-dimension/tree/master/skdim)
* 5) Apply PCA and visualize the dataset in the space of two first principal components
* 6) Investigate higher-order principal components, try to conclude if they contain useful information
* 7) Apply any non-linear manifold learning method and visualize the dataset
* 8) Apply two or more different clustering methods using clustering quality criteria, compare the results as
* well as computational time
* 9) Apply any matrix factorization method which is not PCA, compare it with PCA as well as computational time

In [ ]:
## Q.1 Import dataset

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving dentate_gyrus_sample3000.csv to dentate_gyrus_sample3000 (2).csv


In [ ]:
dataframe = pd.read_csv(io.StringIO(uploaded["dentate_gyrus_sample3000.csv"].decode("utf-8")))
dataframe.head()

,label,Tmsb10,Apoe,Mt1,Stmn2,Tuba1a,Stmn1,Meg3,Ptn,Mt3,Tubb2b,Fabp7,Cst3,Xist,Sox11,Tubb3,Pcp4,Igfbpl1,Slc1a2,Slc1a3,Tubb5,Snca,Plp1,Atp1a2,Dbi,Nnat,Cd24a,Mt2,Ppp3ca,Ncdn,Gap43,Mllt11,Marcksl1,Aldoc,Camk2a,Plppr4,Clu,Glul,Ptprz1,Sparcl1,...,Shf,2610037D02Rik,Atxn7l2,Tmem74b,Adnp2,Arsg,Esco2,Tmtc3,Rnmtl1,Pald1,Dcp1a,Gemin2,Nckipsd,Notch3,Ubtd1,Simc1,Nkrf,Gabpb2,Fbxo10,Cenpl,Wdr36,Invs,Trim39,Ttc21b,Penk,Zfp629,D130020L05Rik,Pnpt1,Ice2,Pifo,Nol10,Gm6356,Tpmt,Gba,Slc25a20,Col19a1,Zfp516,Rgp1,Col26a1,Jpx
0,Neuroblast,4.47,0.00,0.35,3.09,4.62,3.72,4.04,0.19,0.19,3.81,0.19,1.43,3.4,2.30,3.01,0.00,0.99,0.00,0.00,3.63,1.58,0.00,0.00,0.49,3.83,2.08,0.19,1.62,0.35,1.88,3.01,3.59,0.19,0.00,0.62,0.00,0.72,0.35,0.00,...,0.0,0.00,0.00,0.19,0.00,0.00,0.0,0.0,0.0,0.0,0.19,0.19,0.0,0.00,0.19,0.0,0.19,0.00,0.19,0.00,0.00,0.19,0.0,0.0,0.0,0.19,0.00,0.00,0.35,0.0,0.19,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.49,0.00
1,Immature-GC,2.36,0.36,0.84,1.40,1.76,0.84,2.36,0.00,2.65,1.29,0.36,1.29,0.0,1.40,0.63,3.27,0.00,1.01,0.00,0.84,2.96,1.01,0.00,0.63,0.63,0.00,0.36,3.54,2.32,0.00,2.68,2.08,0.00,2.44,2.51,0.00,0.00,0.63,1.29,...,0.0,0.00,0.00,0.00,0.63,0.00,0.0,0.0,0.0,0.0,0.00,0.36,0.0,0.00,0.63,0.0,0.00,0.00,0.36,0.00,0.63,0.00,0.0,0.0,0.0,0.00,0.36,0.36,0.00,0.0,0.36,0.0,0.0,0.63,0.00,0.0,0.0,0.36,0.00,0.00
2,GC-juv,1.71,5.08,2.98,0.52,2.95,1.31,0.75,3.97,3.32,1.57,4.53,4.22,0.0,1.18,0.37,0.20,0.75,1.62,3.15,1.37,0.37,0.20,1.92,4.98,2.72,0.37,2.41,1.37,0.00,0.00,0.75,2.23,2.82,0.00,0.52,2.28,1.75,2.35,2.87,...,0.0,0.00,0.00,0.00,0.20,0.00,0.0,0.0,0.0,0.2,0.20,0.00,0.2,0.20,0.00,0.2,0.00,0.00,0.37,0.20,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.2,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.20,0.00,0.00
3,CA3-Pyr,4.29,0.20,0.20,3.35,4.26,3.88,3.82,1.24,0.00,3.52,0.37,1.47,1.1,1.10,2.85,3.23,1.70,0.20,0.00,4.03,1.02,0.00,0.00,1.61,2.90,2.10,0.00,1.17,0.37,3.10,2.84,3.09,0.00,0.00,0.00,0.20,0.51,0.20,0.00,...,0.2,0.20,0.20,0.00,0.00,0.20,0.0,0.2,0.2,0.0,0.00,0.00,0.0,0.00,0.20,0.2,0.00,0.00,0.00,0.00,0.20,0.00,0.2,0.2,0.0,0.00,0.00,0.20,0.00,0.0,0.00,0.2,0.0,0.00,0.20,0.0,0.0,0.00,0.37,0.00
4,GC-juv,1.91,4.08,3.24,0.41,2.67,1.01,1.10,4.69,3.58,1.79,5.36,4.31,0.0,0.41,0.22,1.18,0.22,2.44,3.25,1.79,0.92,0.22,2.83,4.38,1.39,0.81,2.82,1.18,0.00,0.56,0.41,2.44,2.77,0.00,0.22,1.10,2.58,2.83,2.98,...,0.0,0.22,0.22,0.22,0.00,0.22,0.0,0.0,0.0,0.0,0.22,0.00,0.0,0.22,0.41,0.0,0.00,0.22,0.00,0.22,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.22,0.22,0.0,0.0,0.00,0.00,0.41


In [ ]:
dataframe.to_pickle(path="/content/drive/MyDrive/Master 5A/FML/brain_cell_df.pkl")

In [ ]:
print(f'Dataset has {dataframe.shape[0]} cells and {dataframe.shape[1]} features')

Dataset has 3000 cells and 10001 features


In [ ]:
dataframe.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Columns: 10001 entries, label to Jpx
dtypes: float64(10000), object(1)
memory usage: 228.9+ MB


In [58]:
print("Number of categories of cell: ",len(dataframe['label'].value_counts()))
dataframe['label'].value_counts()

Number of categories of cell:  24


Immature-Pyr            589
GC-juv                  429
Immature-GC             319
GC-adult                304
Neuroblast              178
Astro-adult             151
Immature-GABA           121
Astro-juv               104
RGL_young                85
MOL                      82
OPC                      81
Immature-Astro           75
Endothelial              74
CA3-Pyr                  65
Cajal-Retzius            61
nIPC-perin               57
MiCajal-Retziusoglia     50
nIPC                     40
NFOL                     33
RGL                      25
VLMC                     22
GABA                     22
Ependymal                19
PVM                      14
Name: label, dtype: int64

# Preparing data

### Q.2 Check if you have missing values and suggest a strategy for imputing them

Helful link: `https://towardsdatascience.com/6-different-ways-to-compensate-for-missing-values-data-imputation-with-examples-6022d9ca0779`

We can either check if there is any missing value for each cell by searching NaN per row or check if there is any missing value per features by checking each row.

It depends on what we want for our dataset. If there are features with a lot of missing elements, we can either impute values or choose to delete that features. As 10 000 is a huge number of features, and we will work on reducing its number, if one feature is missing, we can neglect it (well if we consider each feature having the same weight importance). 

Another solution would be to imput data, however it requires some domain experience.


In [59]:
# Percentages of missing values
print((dataframe.isnull().mean() * 100).sort_values(ascending = True))

label      0.0
Fam135a    0.0
Abcf2      0.0
Mfsd11     0.0
Ascc1      0.0
          ... 
Dbndd2     0.0
Map2k4     0.0
Chmp2b     0.0
Rassf8     0.0
Jpx        0.0
Length: 10001, dtype: float64


There is no missing values in our dataset.


**What we can say:**

There are three main types of missing data:
- Missing completely at random (MCAR) : These are the missing data points that follow no discernable pattern.
- Missing at random (MAR) : The missing value can roughly be interpolated from the remaining values to a reasonable degree of accuracy.  If conditioning on another feature increases the likelihood of a particular value compared to a proportional distribution, this value is likely MAR.
- Not missing at random (NMAR): data where the mechanism for why the data is missing is known. Still, the values can not effectively be inferred or predicted.

**How we would replace the missing values:**

Each possible solution depends on the use case. 
- If there is a lot of missing data for one feature, as we have 10000 features and we suppose that each feature has the same importance weight and are not strongly correlated with each other, best would be to drop this feature. Replacing missing data by statistics such as mean, median, max or min would induce bias and won't factor the correlations between features, it only works on the column level. 
- If there is some MCAR data, as each cell belongs to one of the 24 categories, we could group the cells by its categories, and then try to do some imputation using k-NN. The k nearest neighbours is an algorithm that is used for simple classification. The algorithm uses ‘feature similarity’ to predict the values of any new data points. This means that the new point is assigned a value based on how closely it resembles the points in the training set. However it can be quite sensitive to outliers in the data.
- We can also work with the MICE procedure: MICE is a multiple imputation method used to replace missing data values in a data set under certain assumptions about the data missingness mechanism (e.g., the data are missing at random, the data are missing completely at random).


### Q.3 Check if the dataset needs to be normalized in a particular way

Helpful link: 
- `https://towardsdatascience.com/how-to-normalize-single-cell-a438281ea654`
- `https://towardsdatascience.com/tagged/stats-ml-life-sciences`
- `https://machinelearningmastery.com/a-gentle-introduction-to-normality-tests-in-python/`
- `https://towardsai.net/p/data-science/how-when-and-why-should-you-normalize-standardize-rescale-your-data-3f083def38ff`
- `https://www.machinecurve.com/index.php/2020/11/23/feature-scaling-with-python-and-sparse-data/`

In [60]:
# Does the data need to be normalized? 

dataframe.iloc[:,1:].max().sort_values(ascending=False)

Malat1     7.70
Plp1       7.44
Cst3       7.29
Ttr        7.17
Ptgds      7.15
           ... 
Ppwd1      0.76
Nle1       0.76
Tmem74b    0.75
Gba        0.75
Pgm3       0.72
Length: 10000, dtype: float64

In [61]:
col_ranges = dataframe.iloc[:,1:].max() - dataframe.iloc[:,1:].min() # Looking at the data range for each feature
print((col_ranges.sort_values(ascending=False)))

Plp1       7.44
Malat1     7.33
Cst3       7.29
Ttr        7.17
Ptgds      7.15
           ... 
Ppwd1      0.76
Nle1       0.76
Tmem74b    0.75
Gba        0.75
Pgm3       0.72
Length: 10000, dtype: float64


In [62]:
dataframe.iloc[:,1:].std() # Checking the standard variation of our data

Tmsb10     1.662372
Apoe       1.469267
Mt1        1.467278
Stmn2      1.436146
Tuba1a     1.419872
             ...   
Col19a1    0.131733
Zfp516     0.139209
Rgp1       0.144421
Col26a1    0.142950
Jpx        0.140936
Length: 10000, dtype: float64

We can see that features in our dataset have different range, going from 0.72 for Pgm3 (the smallest range) to 7.44 for Plp1. It is quite large (x10 factor) and implies that some features will definitly outweight others. Even the standard variation difference between data is important.

There are plenty of methods to "normalize our data" but the two principals are: either we normalize it, or we standardize it. 

- Standard Scaler- Transforms features independently to unit variance and zero centered.Its data value range is fixed between 0 and 1 . Most commonly used.
- MInMax Scaler - Alternate to standard scaling which has agility to set the minimum and maximum range of data value. e.g. -1 to +1, -10 to +10
Min max scaler should be used when it is required to capture small variance in features and also for sparse data where zero value needs to be preserved.

In [69]:
dataframe.head()

,label,Tmsb10,Apoe,Mt1,Stmn2,Tuba1a,Stmn1,Meg3,Ptn,Mt3,Tubb2b,Fabp7,Cst3,Xist,Sox11,Tubb3,Pcp4,Igfbpl1,Slc1a2,Slc1a3,Tubb5,Snca,Plp1,Atp1a2,Dbi,Nnat,Cd24a,Mt2,Ppp3ca,Ncdn,Gap43,Mllt11,Marcksl1,Aldoc,Camk2a,Plppr4,Clu,Glul,Ptprz1,Sparcl1,...,Shf,2610037D02Rik,Atxn7l2,Tmem74b,Adnp2,Arsg,Esco2,Tmtc3,Rnmtl1,Pald1,Dcp1a,Gemin2,Nckipsd,Notch3,Ubtd1,Simc1,Nkrf,Gabpb2,Fbxo10,Cenpl,Wdr36,Invs,Trim39,Ttc21b,Penk,Zfp629,D130020L05Rik,Pnpt1,Ice2,Pifo,Nol10,Gm6356,Tpmt,Gba,Slc25a20,Col19a1,Zfp516,Rgp1,Col26a1,Jpx
0,Neuroblast,4.47,0.00,0.35,3.09,4.62,3.72,4.04,0.19,0.19,3.81,0.19,1.43,3.4,2.30,3.01,0.00,0.99,0.00,0.00,3.63,1.58,0.00,0.00,0.49,3.83,2.08,0.19,1.62,0.35,1.88,3.01,3.59,0.19,0.00,0.62,0.00,0.72,0.35,0.00,...,0.0,0.00,0.00,0.19,0.00,0.00,0.0,0.0,0.0,0.0,0.19,0.19,0.0,0.00,0.19,0.0,0.19,0.00,0.19,0.00,0.00,0.19,0.0,0.0,0.0,0.19,0.00,0.00,0.35,0.0,0.19,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.49,0.00
1,Immature-GC,2.36,0.36,0.84,1.40,1.76,0.84,2.36,0.00,2.65,1.29,0.36,1.29,0.0,1.40,0.63,3.27,0.00,1.01,0.00,0.84,2.96,1.01,0.00,0.63,0.63,0.00,0.36,3.54,2.32,0.00,2.68,2.08,0.00,2.44,2.51,0.00,0.00,0.63,1.29,...,0.0,0.00,0.00,0.00,0.63,0.00,0.0,0.0,0.0,0.0,0.00,0.36,0.0,0.00,0.63,0.0,0.00,0.00,0.36,0.00,0.63,0.00,0.0,0.0,0.0,0.00,0.36,0.36,0.00,0.0,0.36,0.0,0.0,0.63,0.00,0.0,0.0,0.36,0.00,0.00
2,GC-juv,1.71,5.08,2.98,0.52,2.95,1.31,0.75,3.97,3.32,1.57,4.53,4.22,0.0,1.18,0.37,0.20,0.75,1.62,3.15,1.37,0.37,0.20,1.92,4.98,2.72,0.37,2.41,1.37,0.00,0.00,0.75,2.23,2.82,0.00,0.52,2.28,1.75,2.35,2.87,...,0.0,0.00,0.00,0.00,0.20,0.00,0.0,0.0,0.0,0.2,0.20,0.00,0.2,0.20,0.00,0.2,0.00,0.00,0.37,0.20,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.2,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.20,0.00,0.00
3,CA3-Pyr,4.29,0.20,0.20,3.35,4.26,3.88,3.82,1.24,0.00,3.52,0.37,1.47,1.1,1.10,2.85,3.23,1.70,0.20,0.00,4.03,1.02,0.00,0.00,1.61,2.90,2.10,0.00,1.17,0.37,3.10,2.84,3.09,0.00,0.00,0.00,0.20,0.51,0.20,0.00,...,0.2,0.20,0.20,0.00,0.00,0.20,0.0,0.2,0.2,0.0,0.00,0.00,0.0,0.00,0.20,0.2,0.00,0.00,0.00,0.00,0.20,0.00,0.2,0.2,0.0,0.00,0.00,0.20,0.00,0.0,0.00,0.2,0.0,0.00,0.20,0.0,0.0,0.00,0.37,0.00
4,GC-juv,1.91,4.08,3.24,0.41,2.67,1.01,1.10,4.69,3.58,1.79,5.36,4.31,0.0,0.41,0.22,1.18,0.22,2.44,3.25,1.79,0.92,0.22,2.83,4.38,1.39,0.81,2.82,1.18,0.00,0.56,0.41,2.44,2.77,0.00,0.22,1.10,2.58,2.83,2.98,...,0.0,0.22,0.22,0.22,0.00,0.22,0.0,0.0,0.0,0.0,0.22,0.00,0.0,0.22,0.41,0.0,0.00,0.22,0.00,0.22,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.22,0.22,0.0,0.0,0.00,0.00,0.41


In [92]:
# Count the number of zero per column to see how sparse our dataset is (%)
print(dataframe.iloc[:, 1:].astype(bool).sum(axis=0) / dataframe.shape[0] *100)

# Check the sparsity of our matrix
sparr = dataframe.iloc[:, 1:].to_numpy()
print("sparsity:", (sparr == 0).mean())

Tmsb10     94.633333
Apoe       87.700000
Mt1        90.766667
Stmn2      81.400000
Tuba1a     99.200000
             ...    
Col19a1    13.533333
Zfp516     24.866667
Rgp1       34.266667
Col26a1    14.000000
Jpx        31.733333
Length: 10000, dtype: float64
sparsity: 0.39000076666666667


Our dataset is quite sparse (40% of the data is equals to 0).

In [ ]:
df.astype(bool).sum(axis=0)

In [93]:
# Separating labels from data
cell_labels = dataframe["label"].copy()
df = dataframe.iloc[:, 1:].copy()


# Standardizing our data
scaler = StandardScaler() 
data_scaled = scaler.fit_transform(df)

In [94]:
print("mean:", data_scaled.mean(axis=0))
print("std:", data_scaled.std(axis=0))

mean: [ 3.11454566e-16 -2.81256500e-17 -5.74355378e-17 ... -5.86197757e-17
  7.10542736e-18 -6.33567273e-17]
std: [1. 1. 1. ... 1. 1. 1.]


We decided to use Standard Scalar over Min max as it will not lower the variance like MinMax (useful when using PCA). Moreover, we will consider the effect of outliers important, as we want to visualize the data, thus keeping the standard scalar normalization.

However, our dataset is quite sparse (around 40% data). Using traditional scaling methods aren't the best solution in this case. Indeed, both of these methods will make the center shift and will "break" the sparsity.

We can use instead **MaxAbsScalar**.

Scale each feature by its maximum absolute value. This estimator scales and translates each feature individually such that the maximal absolute value of each feature in the training set will be 1.0. It does not shift/center the data, and thus does not destroy any sparsity.


In [97]:
from sklearn.preprocessing import MaxAbsScaler
scaler = MaxAbsScaler()
data_scaled = scaler.fit_transform(df)
standardized_dataset = scaler.transform(df)
print(standardized_dataset)

[[0.83395522 0.         0.05376344 ... 0.         0.2816092  0.        ]
 [0.44029851 0.05454545 0.12903226 ... 0.47368421 0.         0.        ]
 [0.31902985 0.76969697 0.4577573  ... 0.26315789 0.         0.        ]
 ...
 [0.         0.17575758 0.32104455 ... 0.         0.         0.        ]
 [0.76865672 0.0469697  0.02611367 ... 0.22368421 0.17816092 0.        ]
 [0.92910448 0.02424242 0.         ... 0.21052632 0.         0.        ]]


We are preserving the sparsity of our dataset.

### Q.4 Give an estimate of intrinsic dimensionality of the dataset

### Q.5  Apply PCA and visualize the dataset in the space of two first principal components

### Q.6 Investigate higher-order principal components, try to conclude if they contain useful information

### Q.7 Apply any non-linear manifold learning method and visualize the dataset

### Q.8  Apply two or more different clustering methods using clustering quality criteria, compare the results as well as computational time

### Q.9 Apply any matrix factorization method which is not PCA, compare it with PCA as well as computational time